# 1. Problem

We have an array arr[0 . . . n-1]. We should be able to

1. Find the sum of elements from index l to r where 0 <= l <= r <= n-

2. Change value of a specified element of the array to a new value x. We need to do arr[i] = x where 0 <= i <= n-1.

## I. Simple Solution

1. Run a loop from l to r and calculate the sum of elements in the given range. 
2. To update a value, simply do arr[i] = x. 

The first operation takes O(n) time and the second operation takes O(1) time.

## II. Another Solution

1. Create another array and store sum from start to i at the ith index in this array.

The sum of a given range can now be calculated in O(1) time, but update operation takes O(n) time now. This works well if the number of query operations is large and very few updates.

## III. Segment Tree

We can use a Segment Tree to do both operations in O(Log n) time.

### Representation

- Input: Leaf Node
- Internal node: merging of the leaf nodes

#### Indices at node i

1. Left child node: 2 * i + 1
2. Right child node: 2 * i + 2
3. Parent node: (i - 1) / 2

### Differences

> Like Heap, the segment tree is also represented as an array. The difference here is, it is not a complete binary tree. It is rather a full binary tree (every node has 0 or 2 children) and all levels are filled except possibly the last level. Unlike Heap, the last level may have gaps between nodes. 

> All levels of the constructed segment tree will be completely filled except the last level. Also, the tree will be a Full Binary Tree because we always divide segments in two halves at every level.

### Size

Since the constructed tree is always a full binary tree with n leaves, there will be n-1 internal nodes. If n is a power of 2, then there are no dummy nodes. So the size of the segment tree is 2n-1 (n leaf nodes and n-1) internal nodes. If n is not a power of 2, then the size of the tree will be 2*x – 1 where x is the smallest power of 2 greater than n. For example, when n = 10, then size of array representing segment tree is 2$*$16-1 = 31.

### Height
Height of the segment tree will be $log_2(n)$. Since the tree is represented using array and relation between parent and child indexes must be maintained, size of memory allocated for segment tree will be $2 * 2^(log_2n) - 1$

### Construction

In [13]:
def make_st(arr, idx, low, high, segment_tree):
    if low == high:
        segment_tree[idx] = arr[low]
        return arr[low]
    
    midpoint = low + (high - low) // 2
    
    left_idx = idx * 2 + 1
    right_idx = idx * 2 + 2
    segment_tree[idx] = make_st(arr, left_idx, low, midpoint, segment_tree) + make_st(arr, right_idx, midpoint + 1, high, segment_tree)
    
    return segment_tree[idx]

In [14]:
from math import ceil, log2

if __name__ == "__main__":
    arr = [1, 3, 5, 7, 9, 11]
    n = len(arr)
    
    height = (int)(ceil(log2(n)))
    max_size = 2 * (int)(2**height) - 1
    segment_tree = [0] * max_size
    low = 0
    high = n - 1
    idx = 0
    
    make_st(arr,idx, low, high, segment_tree)

### Sum of range

In [5]:
def find_sum(root, idx, low, high, query_low, query_high):
    if query_low <= low and query_high >= high:
        return root[idx]
    
    if query_low > high or query_high < low:
        return 0
    
    midpoint = low + (high - low) // 2
    left_idx = 2 * idx + 1
    right_idx = 2 * idx + 2
    
    return find_sum(root, left_idx, low, midpoint, query_low, query_high) + find_sum(root, right_idx, midpoint + 1, high, query_low, query_high)

### Update

In [17]:
# i = index in the input array
# diff = value to be added to all nodes which have i in range
# diff = new_val - arr[i]
def update(segment_tree, i, low, high, diff, idx):
    if low < i or i > high:
        return
    
    segment_tree[idx] = segment_tree[idx] + diff
    
    if low != high:
        midpoint = low + (high - low) // 2
        
        idx_left = 2 * i + 1
        idx_right = 2 * i + 2
        update(segment_tree, i, low, midpoint, diff, idx_left)
        update(segment_tree, i, midpoint + 1, high, diff, idx_right)